# Importando as dependencias necessárias 

In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from tqdm import tqdm # verifica o progresso de uma tarefa
from statistics import mean

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier

# instalar pip install pandas-profilling
from pandas_profiling import ProfileReport

## Criando Dataframe e limpeza de dados


#### Motivos de limpeza genericos:
* estado_onde_nasceu - informacao desnecessaria 
* id_solicitante - informação com dados unicos
* grau_instrucao - apenas possui valor 0
* estado_onde_trabalha - muitos dados nulos
* codigo_area_telefone_trabalho - informação desnecessaria
* codigo_area_telefone_residencial - informação desnecessaria
* possui_telefone_celular - apenas possui valor "N"
* local_onde_trabalha - local_onde_reside igual a local_onde_trabalha


#### Os dados abaixo possuem muitos valores nulos:

* grau_instrucao_companheiro - 12860 dados nulos (64.3%) 
* profissao_companheiro - 11514 dados nulos (57.6%)

#### Teste com alguns dados:
Ao remover um desses valores, no modelo RandomForestClassifier, foi observado o seguinte comportamento:
* possui_email - essa informação reduz a acuracia do modelo (~ 0.15%)
* local_onde_reside - essa informação reduz a acuracia do modelo (~0.29%)
Porém ao utilizar esses dois dados juntos, o modelo apresentou uma melhora de acuracia superior a acuracia observado ao remover os dois dados. Então foi decidido manter esses dois atributos.

Ao expandir o atributo `estado_onde_reside` em 27 colunas com valores binarios (0/1) foi observado redução na acuracia do modelo, então esse atributo também será removido, além disso, visualmente os atributos `local_onde_reside` e `estado_onde_reside` possuem uma certa correlação, e como `local_onde_reside` e `possui_email` possuem sinergia, é preferivel manter `local_onde_reside` ao invés de `estado_onde_reside` .

Apos a limpeza sobraram 15.568 de 20.000 ( ~22.16% de remoção)


In [ ]:
df = pd.read_csv('conjunto_de_treinamento.csv')

### Criando arquivo de analise do dataset

Nesta etapa é gerado um arquivo html onde é feita uma varredura nos dados, retornando quais são as possíveis correlações entre as colunas, numero de amostras nulas, repetição de dados, tipo dos dados, análises estatisticas como os quartis, desvio padrão, variância. No final do arquivo há uma matriz de correlação dos dados.

Antes de gerar esse arquivo com o diagnóstico dos dados, foi decidido remover algumas colunas que foram consideradas insignificantes, ou por contribuir negativamente na acurácia do modelo.

a biblioteca "pandas_profiling" é instalada através do comando `pip install pandas-profiling`

In [ ]:
prof = ProfileReport(df)
prof.to_file(output_file='analise_inicial_dados.html')

Excluindo colunas desnecessárias após analizar o dataframe com o profile report, e após testar de outras formas descritas anteriormente. 

In [ ]:
df = df.drop(['estado_onde_nasceu',  #informacao desnecessaria 
                    'id_solicitante', 
                    'grau_instrucao',  # apenas possui valor 0 
                    'estado_onde_trabalha',
                    'codigo_area_telefone_trabalho',
                    'codigo_area_telefone_residencial',
                    'possui_telefone_celular',  # apenas possui valor "N" 
                    'local_onde_trabalha',  #local_onde_reside igual a local_onde_trabalha
                    'estado_onde_reside',                    
                    'grau_instrucao_companheiro',
                    'profissao_companheiro',
                    
                   ],axis=1)


Analisando dataframe após a exclusao dos dados

In [ ]:
prof = ProfileReport(df)
prof.to_file(output_file='analise_apos_limpeza.html')

## Carregando conjunto de testes

In [ ]:
df_teste = pd.read_csv('conjunto_de_teste.csv')

df_teste = df_teste.drop(['estado_onde_nasceu',  #informacao desnecessaria 
#                     'id_solicitante', 
                    'grau_instrucao',  # apenas possui valor 0 
                    'estado_onde_trabalha',
                    'codigo_area_telefone_trabalho',
                    'codigo_area_telefone_residencial',
                    'possui_telefone_celular',  # apenas possui valor "N" 
                    'local_onde_trabalha',  #local_onde_reside igual a local_onde_trabalha
                    'estado_onde_reside',
                    
#                     'possui_email',                     
                    'grau_instrucao_companheiro',
                    'profissao_companheiro',
#                     'local_onde_reside',
#                     'forma_envio_solicitacao',
#                     'sexo'
                    
                   ],axis=1)

## Definindo variaveis categoricas e binarizando dados



No conjunto de treino foi decidido excluir os dados que apresentam nulos

In [ ]:
variaveis_categoricas = [
    x for x in df.columns if df[x].dtype == 'object' or x == 'area_code'
    ]

df = pd.get_dummies(df,columns=[
                                'forma_envio_solicitacao',
                                'sexo'
                               ])

binarizador = LabelBinarizer()
for v in ['possui_telefone_residencial',
        'vinculo_formal_com_empresa','possui_telefone_trabalho']:
    df[v] = binarizador.fit_transform(df[v])
    
# df = df.dropna()

# df['profissao'] = df['profissao'].fillna(df['profissao'].mode()[0])
df['profissao'] = df['profissao'].fillna(df['profissao'].mode()[0])
df['ocupacao'] = df['ocupacao'].fillna(df['ocupacao'].mode()[0])
df['meses_na_residencia'] = df['meses_na_residencia'].fillna(df['meses_na_residencia'].mode()[0])


No conjunto de teste os valores nulos foram substituidos pelos valores de moda

In [ ]:
variaveis_categoricas = [
    x for x in df_teste.columns if df_teste[x].dtype == 'object' or x == 'area_code'
    ]

df_teste = pd.get_dummies(df_teste,columns=[
                                'forma_envio_solicitacao',
                                'sexo'
                               ])

binarizador = LabelBinarizer()
for v in ['possui_telefone_residencial',
        'vinculo_formal_com_empresa','possui_telefone_trabalho']:
    df_teste[v] = binarizador.fit_transform(df_teste[v])
    

df_teste['profissao'] = df_teste['profissao'].fillna(df_teste['profissao'].mode()[0])
df_teste['ocupacao'] = df_teste['ocupacao'].fillna(df_teste['ocupacao'].mode()[0])
df_teste['meses_na_residencia'] = df_teste['meses_na_residencia'].fillna(df_teste['meses_na_residencia'].mode()[0])


In [ ]:
atributos_selecionados = [
'sexo_ ',
'sexo_M',
'sexo_F',
'sexo_N',
'possui_email',
'local_onde_reside',
'tipo_endereco',
'idade',
'estado_civil',
'qtde_dependentes',
'dia_vencimento',  
'possui_telefone_residencial',   
'meses_na_residencia',
'renda_mensal_regular',
'renda_extra',
'possui_cartao_visa',
'possui_cartao_mastercard',  
'possui_outros_cartoes',
'qtde_contas_bancarias',
'qtde_contas_bancarias_especiais',
'valor_patrimonio_pessoal',
'possui_carro',
'vinculo_formal_com_empresa',  
'meses_no_trabalho',
'profissao',
'ocupacao',
'forma_envio_solicitacao_correio',
'forma_envio_solicitacao_internet',
'forma_envio_solicitacao_presencial',
    
'inadimplente'
]

dados_selecionados = df[atributos_selecionados]

dados_embaralhados = dados_selecionados.sample(frac=1,random_state=12345)

x = dados_embaralhados.loc[:,dados_embaralhados.columns!='inadimplente'].values
y = dados_embaralhados.loc[:,dados_embaralhados.columns=='inadimplente'].values

x_treino, x_teste, y_treino, y_teste = train_test_split(
    x,
    y.ravel(),
    test_size=0.01,
    shuffle=True,
    random_state=777
    )

ajustador_de_escala = MinMaxScaler()
ajustador_de_escala.fit(x_treino)

x_treino = ajustador_de_escala.transform(x_treino)
x_teste  = ajustador_de_escala.transform(x_teste)


## Comparando e avaliando diferentes modelos

Após treinar alguns modelos, foi decidido que RandomForest e GradientBoosting foram os que apresentaram melhores resultados.
E baseado em algumas pesquisas:

* https://www.datasciencecentral.com/profiles/blogs/decision-tree-vs-random-forest-vs-boosted-trees-explained
        
* https://www.datacamp.com/community/tutorials/random-forests-classifier-python?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=278443377095&utm_targetid=aud-392016246653:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1001655&gclid=CjwKCAjwm7mEBhBsEiwA_of-TMa9e-x8s99xu1TOGVHuT24xXCHgq8znmeG7FCvVcqRE3k-emNXirRoC8OUQAvD_BwE

## RandomForestClassifier

criterion='entropy'

scores:[0.59152216 0.58381503 0.58831085 0.59216442 0.58574181 0.58445729
 0.59922929 0.59666024 0.57133676 0.58033419] acurácia média = 58.74 %

criterion='gini'
scores:[0.58702633 0.58124599 0.58702633 0.5973025  0.59344894 0.58124599
 0.59666024 0.59152216 0.57455013 0.58226221] acurácia média = 58.72 %
 

Após realizar alguns experimentos alterando o valor de  max_depth mantendo o mesmo random_state, foi notado que o valor da acuracia aumenta, porém quando max_depth é maior que 4 os valores começam a se comportar de forma aleatoria em torno do maximo observado em max_depth=4, assim foi decidido manter max_depth=4.
 

In [ ]:
clf = RandomForestClassifier(
    max_depth=4, 
#                              random_state=0, 
                             criterion='entropy', 
#                              max_features='log2',
                             min_samples_split=2, 
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0)
# clf.fit(x_treino, y_treino)

q = 10
scores = cross_val_score(
        clf,
        x_treino,
        y_treino.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
#     f"numero de estimadore: {k}",
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

In [ ]:
acuracias = []
for k in tqdm(range(1,10,1)):
#     print(f"numero de estimadore: {k}")
    clf = RandomForestClassifier(max_depth=k, 
                                 random_state=0, 
                                 criterion='entropy', 
    #                              max_features='log2',
                                 min_samples_split=2, 
                                 min_samples_leaf=1,
                                 min_weight_fraction_leaf=0.0)
    # clf.fit(x_treino, y_treino)

    q = 10
    scores = cross_val_score(
            clf,
            x,
            y.ravel(),
            cv=q
            )
    acuracias.append(round(100*mean(scores),2))

    print (
    #         f'k = {k}',
        f"numero de estimadore: {k}",
            f'scores:{scores}',
            f'acurácia média = {round(100*mean(scores),2)} %' 
            )
print(f"acuracias: {acuracias}")
print(f"media acuracias: {mean(acuracias)}")


    # print(clf.predict(x_teste))



Variando o valor de max_depth
 
 5%|████▎                                                                              | 1/19 [00:03<01:05,  3.63s/it]
numero de estimadore: 1 scores:[0.56775851 0.57353886 0.55234425 0.57867694 0.57096981 0.55362877
 0.57353886 0.55427103 0.55655527 0.57005141] acurácia média = 56.51 %
 11%|████████▋                                                                          | 2/19 [00:07<01:08,  4.05s/it]
numero de estimadore: 2 scores:[0.58509955 0.57353886 0.58317277 0.58317277 0.57418112 0.56390495
 0.58509955 0.57418112 0.5655527  0.5655527 ] acurácia média = 57.53 %
 16%|█████████████                                                                      | 3/19 [00:13<01:14,  4.65s/it]
numero de estimadore: 3 scores:[0.58831085 0.5761079  0.57482338 0.58831085 0.56647399 0.58124599
 0.58509955 0.58702633 0.56426735 0.57712082] acurácia média = 57.89 %
 21%|█████████████████▍                                                                 | 4/19 [00:19<01:20,  5.35s/it]
numero de estimadore: 4 scores:[0.58959538 0.58060373 0.57675016 0.59023764 0.58638407 0.57739242
 0.59987155 0.59152216 0.57455013 0.58097686] acurácia média = 58.48 %
 26%|█████████████████████▊                                                             | 5/19 [00:26<01:23,  5.99s/it]
numero de estimadore: 5 scores:[0.58702633 0.58188825 0.58959538 0.58766859 0.58509955 0.5793192
 0.59537572 0.59280668 0.57712082 0.58611825] acurácia média = 58.62 %
 32%|██████████████████████████▏                                                        | 6/19 [00:34<01:27,  6.71s/it]
numero de estimadore: 6 scores:[0.5940912  0.57996146 0.58831085 0.58831085 0.58317277 0.57096981
 0.59858703 0.58959538 0.5751928  0.57840617] acurácia média = 58.47 %
 37%|██████████████████████████████▌                                                    | 7/19 [00:44<01:31,  7.60s/it]
numero de estimadore: 7 scores:[0.58831085 0.58895311 0.59023764 0.59023764 0.58831085 0.57675016
 0.59023764 0.58702633 0.57583548 0.5777635 ] acurácia média = 58.54 %
 42%|██████████████████████████████████▉                                                | 8/19 [00:55<01:34,  8.61s/it]
numero de estimadore: 8 scores:[0.5908799  0.58766859 0.58831085 0.59216442 0.58445729 0.58253051
 0.59794477 0.58895311 0.58161954 0.58804627] acurácia média = 58.83 %
 47%|███████████████████████████████████████▎                                           | 9/19 [01:06<01:33,  9.37s/it]
numero de estimadore: 9 scores:[0.58638407 0.58831085 0.58445729 0.5908799  0.5793192  0.58253051
 0.5940912  0.59023764 0.57262211 0.57262211] acurácia média = 58.41 %
 53%|███████████████████████████████████████████▏                                      | 10/19 [01:18<01:31, 10.21s/it]
numero de estimadore: 10 scores:[0.5940912  0.59858703 0.58702633 0.59922929 0.57546564 0.58253051
 0.59152216 0.60115607 0.56619537 0.57712082] acurácia média = 58.73 %
 58%|███████████████████████████████████████████████▍                                  | 11/19 [01:30<01:26, 10.87s/it]
numero de estimadore: 11 scores:[0.58574181 0.60372511 0.58574181 0.58959538 0.58638407 0.58124599
 0.58574181 0.59601798 0.57712082 0.57583548] acurácia média = 58.67 %
 63%|███████████████████████████████████████████████████▊                              | 12/19 [01:44<01:21, 11.63s/it]
numero de estimadore: 12 scores:[0.58831085 0.60115607 0.57739242 0.59344894 0.58124599 0.57353886
 0.57546564 0.58060373 0.57455013 0.59318766] acurácia média = 58.39 %
 68%|████████████████████████████████████████████████████████                          | 13/19 [01:58<01:14, 12.38s/it]
numero de estimadore: 13 scores:[0.58831085 0.59666024 0.57739242 0.60308285 0.57675016 0.57032755
 0.57096981 0.59344894 0.57583548 0.5777635 ] acurácia média = 58.31 %
 74%|████████████████████████████████████████████████████████████▍                     | 14/19 [02:13<01:05, 13.14s/it]
numero de estimadore: 14 scores:[0.58381503 0.58831085 0.56518947 0.59922929 0.56583173 0.58188825
 0.56840077 0.58702633 0.5655527  0.57712082] acurácia média = 57.82 %
 79%|████████████████████████████████████████████████████████████████▋                 | 15/19 [02:30<00:57, 14.29s/it]
numero de estimadore: 15 scores:[0.57739242 0.60308285 0.56647399 0.58766859 0.57225434 0.58060373
 0.56390495 0.5793192  0.56041131 0.57647815] acurácia média = 57.68 %
 84%|█████████████████████████████████████████████████████████████████████             | 16/19 [02:48<00:46, 15.60s/it]
numero de estimadore: 16 scores:[0.57225434 0.60051381 0.55876686 0.5793192  0.57482338 0.57546564
 0.5728966  0.57096981 0.56876607 0.56619537] acurácia média = 57.4 %
 89%|█████████████████████████████████████████████████████████████████████████▎        | 17/19 [03:07<00:33, 16.71s/it]
numero de estimadore: 17 scores:[0.57096981 0.58509955 0.56454721 0.5728966  0.56518947 0.5728966
 0.57867694 0.59023764 0.5655527  0.57197943] acurácia média = 57.38 %
 95%|█████████████████████████████████████████████████████████████████████████████▋    | 18/19 [03:25<00:16, 16.99s/it]
numero de estimadore: 18 scores:[0.58124599 0.59023764 0.55362877 0.58831085 0.56711625 0.56390495
 0.56518947 0.57096981 0.56426735 0.56683805] acurácia média = 57.12 %
100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [03:43<00:00, 11.78s/it]
numero de estimadore: 19 scores:[0.56775851 0.59152216 0.55619782 0.57096981 0.56775851 0.56326268
 0.56711625 0.57675016 0.57069409 0.57262211] acurácia média = 57.05 %
 
 
 valor de max_depth constante e igual a 5, observando a aleatoriedade dos dados:
 
  11%|█████████▎                                                                          | 1/9 [00:07<00:56,  7.06s/it]
numero de estimadore: 1 scores:[0.59601798 0.58253051 0.58638407 0.58959538 0.58766859 0.5793192
 0.59473346 0.58831085 0.57647815 0.58419023] acurácia média = 58.65 %
 22%|██████████████████▋                                                                 | 2/9 [00:14<00:49,  7.06s/it]
numero de estimadore: 2 scores:[0.58702633 0.58445729 0.58188825 0.59601798 0.58959538 0.58317277
 0.59601798 0.58445729 0.57197943 0.57969152] acurácia média = 58.54 %
 33%|████████████████████████████                                                        | 3/9 [00:20<00:41,  6.96s/it]
numero de estimadore: 3 scores:[0.58831085 0.58381503 0.58574181 0.58638407 0.58317277 0.57739242
 0.59152216 0.58317277 0.57390746 0.58161954] acurácia média = 58.35 %
 44%|█████████████████████████████████████▎                                              | 4/9 [00:27<00:34,  6.92s/it]
numero de estimadore: 4 scores:[0.59473346 0.58060373 0.58831085 0.60115607 0.59023764 0.57803468
 0.60436737 0.58895311 0.5751928  0.58997429] acurácia média = 58.92 %
 56%|██████████████████████████████████████████████▋                                     | 5/9 [00:34<00:27,  6.92s/it]
numero de estimadore: 5 scores:[0.5973025  0.58959538 0.58188825 0.58702633 0.58702633 0.58381503
 0.59216442 0.59601798 0.57647815 0.58868895] acurácia média = 58.8 %
 67%|████████████████████████████████████████████████████████                            | 6/9 [00:41<00:20,  6.89s/it]
numero de estimadore: 6 scores:[0.59280668 0.58574181 0.58188825 0.59023764 0.58124599 0.58381503
 0.58959538 0.59601798 0.57583548 0.57840617] acurácia média = 58.56 %
 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [00:48<00:13,  6.86s/it]
numero de estimadore: 7 scores:[0.59216442 0.58638407 0.58895311 0.58574181 0.58702633 0.58253051
 0.59344894 0.59023764 0.58226221 0.5874036 ] acurácia média = 58.76 %
 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [00:55<00:06,  6.86s/it]
numero de estimadore: 8 scores:[0.59280668 0.58188825 0.59216442 0.59280668 0.58895311 0.58253051
 0.5973025  0.58574181 0.57647815 0.58161954] acurácia média = 58.72 %
100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:02<00:00,  6.92s/it]
numero de estimadore: 9 scores:[0.58766859 0.57996146 0.58381503 0.5908799  0.58574181 0.57418112
 0.59344894 0.58959538 0.57840617 0.58161954] acurácia média = 58.45 %
acuracias: [58.65, 58.54, 58.35, 58.92, 58.8, 58.56, 58.76, 58.72, 58.45]
media acuracias: 58.638888888888886


Em 10 experimentos observou-se que para um valor de max_depth=5 os valores de acuracia apresentaram uma media de 58.63%

## GradientBoostingClassifier

Atributos utilizados

scores:[0.60950546 0.59152216 0.59794477 0.60436737 0.58445729 0.58702633
 0.60051381 0.59280668 0.58161954 0.58868895]   
 acurácia média = 59.38 %

atributos_selecionados=[
'sexo_ ',  
'sexo_M',  
'sexo_F',  
'sexo_N',  
'possui_email',  
'local_onde_reside',  
'tipo_endereco',  
'idade',  
'estado_civil',  
'qtde_dependentes',  
'dia_vencimento',  
'possui_telefone_residencial',   
'meses_na_residencia',  
'renda_mensal_regular',  
'renda_extra',  
'possui_cartao_visa',  
'possui_cartao_mastercard',  
'possui_outros_cartoes',  
'qtde_contas_bancarias',  
'qtde_contas_bancarias_especiais',  
'valor_patrimonio_pessoal',  
'possui_carro',  
'vinculo_formal_com_empresa',  
'meses_no_trabalho',  
'profissao',  
'ocupacao',  
'forma_envio_solicitacao_correio',  
'forma_envio_solicitacao_internet',  
'forma_envio_solicitacao_presencial',  
'inadimplente'
] 

#### Verificando acurácia do modelo

In [ ]:
clf = GradientBoostingClassifier(
#     loss='exponential',
#     n_estimators=245, 
    learning_rate=1.0,
     max_depth=1, 
#     random_state=0,
# max_leaf_nodes=10
)

q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

#### Criando o modelo

In [ ]:

clf.fit(x_treino,y_treino.ravel())

Selecionando colunas do conjunto de teste para inserir no modelo criado

In [ ]:
atributos_selecionados = [
'sexo_ ',
'sexo_M',
'sexo_F',
'sexo_N',
'possui_email',
'local_onde_reside',
'tipo_endereco',
'idade',
'estado_civil',
'qtde_dependentes',
'dia_vencimento',  
'possui_telefone_residencial',   
'meses_na_residencia',
'renda_mensal_regular',
'renda_extra',
'possui_cartao_visa',
'possui_cartao_mastercard',  
'possui_outros_cartoes',
'qtde_contas_bancarias',
'qtde_contas_bancarias_especiais',
'valor_patrimonio_pessoal',
'possui_carro',
'vinculo_formal_com_empresa',  
'meses_no_trabalho',
'profissao',
'ocupacao',
'forma_envio_solicitacao_correio',
'forma_envio_solicitacao_internet',
'forma_envio_solicitacao_presencial',
]

testando_modelo = df_teste[atributos_selecionados]

Obtendo a previsão do modelo

In [ ]:
resultado = clf.predict(testando_modelo)

Organizando estrutura do modelo e convertendo para CSV

In [ ]:
dict_resultado = {'id_solicitante': df_teste['id_solicitante'], 'inadimplente': resultado.tolist()}
df_resultado = pd.DataFrame(data=dict_resultado)
df_resultado.to_csv (r'previsao_aprovacao_credito_ramdom_entropy.csv', index = False, header=True)

In [ ]:
prof = ProfileReport(testando_modelo)
prof.to_file(output_file='output_novo_teste.html')

In [ ]:
print(df_teste['id_solicitante'])

In [ ]:
# ids = 

print(len(resultado.tolist()))
print(len(df_teste['id_solicitante']))

d = {'id_solicitante': df_teste['id_solicitante'], 'inadimplente': resultado.tolist()}
df_res = pd.DataFrame(data=d)
print(df_res)

print(type(ids))
print(type(resultado))

# ids['inadimplente'] = resultado.tolist()
# m = ids.insert(0,"inadimplente",resultado )
# print(ids)

In [ ]:

# df_res.to_csv (r'C:\Users\Ron\Desktop\export_dataframe.csv', index = False, header=True)
df_res.to_csv (r'export_dataframe_3.csv', index = False, header=True)

In [ ]:
print(ids.head)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification
# X, y = make_classification(n_samples=1000, n_features=4,
#                             n_informative=2, n_redundant=0,
#                             random_state=0, shuffle=False)

clf = RandomForestClassifier(max_depth=5, 
                             random_state=0, 
                             criterion='entropy', 
                             min_samples_split=2, 
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0)
# clf.fit(x_treino, y_treino)

q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )


# print(clf.predict(x_teste))

In [ ]:
print(x_teste)

In [ ]:
q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

## Teste dos dados de teste
# ------------------------------------------

In [ ]:
clf = clf.fit(x,y.ravel())

#-------------------------------------------------------------------------------
# Obter as respostas do classificador no mesmo conjunto onde foi treinado
#-------------------------------------------------------------------------------
dados = pd.read_csv('conjunto_de_teste.csv')


# y_resposta_treino = classificador.predict(x_treino)



# ----------------------------------------

In [ ]:
import numpy as np
np.std([0.5285806,  0.55234425, 0.50867052,0.49325626,0.54206808, 0.52280026,
 0.50545922,0.55105973,0.54562982,0.56169666])

In [ ]:
clf = RandomForestClassifier(max_depth=5, 
                             random_state=0, 
                             criterion='entropy', 
                             min_samples_split=2, 
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0)
q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

In [ ]:
q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

## Random forest

In [ ]:
clf = RandomForestClassifier(random_state=0)
q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

In [ ]:
clf = RandomForestClassifier(random_state=0)
q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

## Logistic

In [ ]:
q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

## MultinomialNB

In [ ]:
from sklearn.neural_network import MLPClassifier
# for k in range(39,50,2):


In [ ]:
clf = MLPClassifier( max_iter=1000)
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

In [ ]:

for k in range(200,301,20):
    
    print(f" valor de k:{k}")
    
    clf = MLPClassifier( max_iter=k)
    q = 10
    scores = cross_val_score(
            clf,
            x,
            y.ravel(),
            cv=q
            )

    print (
    #         f'k = {k}',
            f'scores:{scores}',
            f'acurácia média = {round(100*mean(scores),2)} %' 
            )

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# lda = LinearDiscriminantAnalysis(n_components=2)
clf = LinearDiscriminantAnalysis(solver='lsqr')

# X_r2 = lda.fit(x, y).transform(x)

q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# lda = LinearDiscriminantAnalysis(n_components=2)
clf = LinearDiscriminantAnalysis(solver='lsqr', )

# X_r2 = lda.fit(x, y).transform(x)

q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(
#     loss='exponential',
    n_estimators=245, 
    learning_rate=1.0,
     max_depth=1, 
#     random_state=0,
# max_leaf_nodes=10
)

q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )



In [ ]:
for k in tqdm(range(245,256,1)):
    print(f"numero de estimadore: {k}")
    
    clf = GradientBoostingClassifier(
    #     loss='exponential',
        n_estimators=k, 
        learning_rate=1.0,
         max_depth=1, 
    #     random_state=0,
    # max_leaf_nodes=10
    )

    q = 10
    scores = cross_val_score(
            clf,
            x,
            y.ravel(),
            cv=q
            )

    print (
    #         f'k = {k}',
            f'scores:{scores}',
            f'acurácia média = {round(100*mean(scores),2)} %' 
            )


In [ ]:
for k in tqdm(range(100,1000,50)):
    print(f"numero de estimadore: {k}")
    clf = GradientBoostingClassifier(
    #     loss='exponential',
        n_estimators=k, 
        learning_rate=1.0,
         max_depth=1, 
    #     random_state=0,
    # max_leaf_nodes=10
    )

    q = 10
    scores = cross_val_score(
            clf,
            x,
            y.ravel(),
            cv=q
            )

    print (
    #         f'k = {k}',
            f'scores:{scores}',
            f'acurácia média = {round(100*mean(scores),2)} %' 
            )

### Teste com GradientBoostingClassifier


De acordo com os resultados abaixo, foi observado que o tempo de execução de treinamento é proporcional ao `n_estimators`, por isso o ideal é manter esse valor o menor possível, o valor com melhor resultado foi de `n_estimators=250`
Acompanhamento dos resultados baseados na alteração do parâmetro `n_estimators`

Após decidir o n_estimators=250 foi feito uma verredura num intervalo mais estreito para validar o valor escolhido, e foi notado que em n_estimators=245 há um ganho de acuracia media.

numero de estimadore: 245  
scores:[0.61014772 0.59344894 0.58766859 0.60950546 0.58060373 0.59344894
 0.59666024 0.60051381 0.57969152 0.58419023] 
acurácia média = 59.36%
 
numero de estimadore: 246
scores:[0.61078998 0.59537572 0.58381503 0.61014772 0.58188825 0.5908799
 0.59601798 0.60179833 0.57840617 0.58290488] acurácia média = 59.32 %
numero de estimadore: 247
 27%|██████████████████████▋                                                            | 3/11 [00:49-02:11, 16.39s/it]
scores:[0.61078998 0.59601798 0.58702633 0.61014772 0.58188825 0.59023764
 0.59344894 0.60115607 0.57969152 0.5848329 ] acurácia média = 59.35 %
numero de estimadore: 248
 36%|██████████████████████████████▏                                                    | 4/11 [01:05-01:54, 16.43s/it]
scores:[0.61078998 0.59473346 0.58574181 0.60950546 0.58124599 0.59280668
 0.59601798 0.60308285 0.57904884 0.58290488] acurácia média = 59.36 %
numero de estimadore: 249
 45%|█████████████████████████████████████▋                                             | 5/11 [01:22-01:38, 16.47s/it]
scores:[0.61078998 0.59473346 0.58702633 0.60950546 0.58188825 0.58895311
 0.59537572 0.59987155 0.57969152 0.5848329 ] acurácia média = 59.33 %
numero de estimadore: 250
 55%|█████████████████████████████████████████████▎                                     | 6/11 [01:38-01:22, 16.55s/it]
scores:[0.61078998 0.5940912  0.58574181 0.6088632  0.57996146 0.5940912
 0.59794477 0.59987155 0.57969152 0.58290488] acurácia média = 59.34 %
numero de estimadore: 251
 64%|████████████████████████████████████████████████████▊                              | 7/11 [01:55-01:06, 16.69s/it]
scores:[0.61078998 0.59537572 0.58574181 0.60950546 0.58124599 0.58831085
 0.59537572 0.60051381 0.58097686 0.58547558] acurácia média = 59.33 %
numero de estimadore: 252
 73%|████████████████████████████████████████████████████████████▎                      | 8/11 [02:12-00:49, 16.67s/it]
scores:[0.61078998 0.59601798 0.58766859 0.6088632  0.58124599 0.59344894
 0.59601798 0.60051381 0.57840617 0.58226221] acurácia média = 59.35 %
numero de estimadore: 253
 82%|███████████████████████████████████████████████████████████████████▉               | 9/11 [02:29-00:33, 16.65s/it]
scores:[0.6088632  0.59666024 0.58574181 0.60500963 0.58124599 0.59023764
 0.59280668 0.60179833 0.58033419 0.58354756] acurácia média = 59.26 %
numero de estimadore: 254
 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [02:46-00:16, 16.77s/it]
scores:[0.6120745  0.59537572 0.58702633 0.60565189 0.57996146 0.59152216
 0.59473346 0.60115607 0.58226221 0.58161954] acurácia média = 59.31 %
numero de estimadore: 255
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [03:03-00:00, 16.67s/it]
scores:[0.6088632  0.59666024 0.59023764 0.60693642 0.58124599 0.58959538
 0.59344894 0.60372511 0.57904884 0.58354756] acurácia média = 59.33 %



----------------------------------------------------------------------------------------------------------------------
 
 
entre 100 e 500 com passo de 50

 0%|                                                                                            | 0/8 [00:00<?, ?it/s]
numero de estimadore: 100
 12%|██████████▌                                                                         | 1/8 [00:07<00:50,  7.16s/it]
scores:[0.61335902 0.58702633 0.58060373 0.59987155 0.58317277 0.58959538
 0.59987155 0.5940912  0.57197943 0.58161954] acurácia média = 59.01 %
numero de estimadore: 150
 25%|█████████████████████                                                               | 2/8 [00:18<00:57,  9.58s/it]
scores:[0.61271676 0.58509955 0.57803468 0.60565189 0.58509955 0.58702633
 0.59344894 0.5908799  0.57390746 0.59061697] acurácia média = 59.02 %
numero de estimadore: 200
 38%|███████████████████████████████▌                                                    | 3/8 [00:32<00:58, 11.77s/it]
scores:[0.61335902 0.58831085 0.57996146 0.6088632  0.57675016 0.58959538
 0.59344894 0.60051381 0.5777635  0.59061697] acurácia média = 59.19 %
numero de estimadore: 250
 50%|██████████████████████████████████████████                                          | 4/8 [00:50<00:56, 14.00s/it]
scores:[0.61078998 0.5940912  0.58574181 0.6088632  0.57996146 0.5940912
 0.59794477 0.59987155 0.57969152 0.58290488] acurácia média = 59.34 %
numero de estimadore: 300
 62%|████████████████████████████████████████████████████▌                               | 5/8 [01:10<00:49, 16.35s/it]
scores:[0.61078998 0.59666024 0.58188825 0.60244059 0.58253051 0.58445729
 0.59473346 0.60115607 0.58033419 0.58290488] acurácia média = 59.18 %
numero de estimadore: 350
 75%|███████████████████████████████████████████████████████████████                     | 6/8 [01:36<00:38, 19.38s/it]
scores:[0.60693642 0.59794477 0.58574181 0.60693642 0.58445729 0.58188825
 0.5973025  0.59537572 0.58097686 0.58354756] acurácia média = 59.21 %
numero de estimadore: 400
 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [02:03<00:22, 22.16s/it]
scores:[0.60629416 0.59794477 0.58445729 0.60822094 0.57996146 0.58124599
 0.59344894 0.59473346 0.58290488 0.58161954] acurácia média = 59.11 %
numero de estimadore: 450
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:34<00:00, 19.27s/it]
scores:[0.60693642 0.59987155 0.58188825 0.60950546 0.58188825 0.58702633
 0.59280668 0.5973025  0.58097686 0.58676093] acurácia média = 59.25 %

## Entre 500 e 100 com passo de 50

0%|                                                                                           | 0/10 [00:00<?, ?it/s]
numero de estimadore: 500
 10%|████████▎                                                                          | 1/10 [00:34<05:13, 34.86s/it]
scores:[0.60308285 0.59987155 0.58702633 0.60950546 0.58317277 0.58188825
 0.59344894 0.60179833 0.5848329  0.58547558] acurácia média = 59.3 %
numero de estimadore: 550
 20%|████████████████▌                                                                  | 2/10 [01:14<05:02, 37.79s/it]
scores:[0.60565189 0.59987155 0.58574181 0.61785485 0.58060373 0.5793192
 0.59023764 0.60051381 0.58161954 0.57969152] acurácia média = 59.21 %
numero de estimadore: 600
 30%|████████████████████████▉                                                          | 3/10 [01:56<04:36, 39.45s/it]
scores:[0.60115607 0.60051381 0.58702633 0.62106615 0.57996146 0.58124599
 0.58831085 0.60244059 0.57840617 0.58161954] acurácia média = 59.22 %
numero de estimadore: 650
 40%|█████████████████████████████████▏                                                 | 4/10 [02:49<04:29, 44.91s/it]
scores:[0.60244059 0.59922929 0.58831085 0.61913937 0.58188825 0.57739242
 0.58766859 0.60436737 0.5751928  0.5874036 ] acurácia média = 59.23 %
numero de estimadore: 700
 50%|█████████████████████████████████████████▌                                         | 5/10 [03:38<03:52, 46.56s/it]
scores:[0.60179833 0.59987155 0.58702633 0.61785485 0.58638407 0.57996146
 0.58702633 0.59987155 0.57390746 0.58676093] acurácia média = 59.2 %
numero de estimadore: 750
 60%|█████████████████████████████████████████████████▊                                 | 6/10 [04:33<03:17, 49.34s/it]
scores:[0.60051381 0.60179833 0.58766859 0.61785485 0.58895311 0.58124599
 0.59152216 0.60051381 0.57583548 0.5874036 ] acurácia média = 59.33 %
numero de estimadore: 800
 70%|██████████████████████████████████████████████████████████                         | 7/10 [05:30<02:35, 51.82s/it]
scores:[0.60051381 0.59922929 0.58831085 0.61592807 0.58574181 0.58060373
 0.59152216 0.59794477 0.57455013 0.58676093] acurácia média = 59.21 %
numero de estimadore: 850
 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [06:29<01:48, 54.17s/it]
scores:[0.59922929 0.59987155 0.58381503 0.61528581 0.58766859 0.57867694
 0.5908799  0.59794477 0.5751928  0.5848329 ] acurácia média = 59.13 %
numero de estimadore: 900
 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [07:42<00:59, 59.83s/it]
scores:[0.59794477 0.60244059 0.58638407 0.61849711 0.58509955 0.57803468
 0.58766859 0.59794477 0.5777635  0.58290488] acurácia média = 59.15 %
numero de estimadore: 950
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [08:50<00:00, 53.01s/it]
scores:[0.59601798 0.60500963 0.58895311 0.61849711 0.58509955 0.5761079
 0.58574181 0.5973025  0.57647815 0.57969152] acurácia média = 59.09 %





In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(
#     loss='exponential',
    n_estimators=100, 
    learning_rate=1.0,
     max_depth=1, 
    random_state=0,
# max_leaf_nodes=10
)

q = 10
scores = cross_val_score(
        clf,
        x,
        y.ravel(),
        cv=q
        )

print (
#         f'k = {k}',
        f'scores:{scores}',
        f'acurácia média = {round(100*mean(scores),2)} %' 
        )
